# Датасет регрессии

## Обоснование выбора

На текущий момент электрокары стремительно входят в использование, особенно в крупных городах, т.к. считаются более экологичной альтернативой - к примеру, в Китае стремительно происходит переход на них. Т.к. батарея электокара составляет основную часть его стоимости, то определение долгосрочности работы батареи является ценной задачей.

## Анализ датасета

# Выбор и обоснование метрик регрессии

# Датасет классификации

## Анализ датасета

# Выбор и обоснование метрик классификации